In [5]:
"""Build a simple weighted minimum edit distance spell corrector using
Dynamic Programming and use it for text5  in nltk.book.
Evaluate your spell corrector. Determine the number of words
from text5  that are present in the dictionary before
and after spell correction. Top 100 Words Spell Correction
Created on Tue Mar 11 10:37:28 2019
@author: Vikas Singh
"""
from pandas import DataFrame, Series
from nltk.corpus import stopwords 
import re
import nltk
from nltk.book import *
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
FltrdText = [w for w in text5 if not w in stop_words] 
FltrdText = [w.lower() for w in FltrdText]
FltrdText = sorted(set(FltrdText))
a=[]
for w in FltrdText:
    if(re.match(('^\w+$'),w)):
        a.append(w)
processed=[]
for w in a:
    x=re.sub(r'\w*\d\w*', '', w).strip()
    if(len(x)>2):
        processed.append(x)
processed= sorted(set(processed))

english_vocab = set(w.lower() for w in nltk.corpus.words.words())
OrgInDic=[]
for i in range(len(processed)):
    OrgInDic.append(processed[i] in english_vocab)
print("Number of Words already Exists in dictionary from Text5",sum(OrgInDic))
print("Number of Words that Not Exists in dictionary from Text5",len(processed)-sum(OrgInDic))
df1=pd.DataFrame(data=[OrgInDic,processed])
df1=df1.transpose()
df1.columns=['OrgInDic','Words']

# Increasing efficiency of our code by adding memory using decorator
#we add some "memory" to our recursive Levenshtein function by adding a dictionary memory 
def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        return func(*args, **kwargs)
    helper.calls = 0
    helper.__name__= func.__name__
    return helper
def memoize(func):
    mem = {}
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in mem:
            mem[key] = func(*args, **kwargs)
        return mem[key]
    return memoizer
@call_counter
@memoize 

####Levenshtein Distance Calculator
# assumption: Weight of the costs for substitions are twice as high as inserts and delets
def levenshtein(s, t, costs=(1, 1, 2)):
    rows = len(s)+1
    cols = len(t)+1
    deletes, inserts, substitutes = costs
    
    dist = [[0 for x in range(cols)] for x in range(rows)]
    # source prefixes can be transformed into empty strings 
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = row * deletes
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = col * inserts
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = substitutes
            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + cost) # substitution
    return dist[row][col]

spell_cor=df1[df1.OrgInDic != True]
ed=[]
wor=[]
i=0
lst=list(english_vocab)
#Top 100 Words Correction
for w in spell_cor.Words:   
    for cw in english_vocab:
        ed.append(levenshtein(cw, w))   
    wor.append(lst[np.argmin(ed)])
    ed=[]
    i=i+1
    if i==100:
        break;
spelldf=pd.DataFrame(wor) 
spelldf = pd.concat([spell_cor.Words[0:100], spelldf], axis=1)
#spelldf = pd.concat([spell_cor.Words, spelldf], axis=1)

Number of Words already Exists in dictionary from Text5 2443
Number of Words that Not Exists in dictionary from Text5 2016


In [6]:
spelldf1=pd.Series(spell_cor.Words[0:100].tolist())
spelldf2=pd.Series(wor) 

results = pd.concat([spelldf1, spelldf2], ignore_index=True, axis=1)
results.columns=['Words', 'SpellCorrected']
print(results)


                Words SpellCorrected
0   aaaaaaaaaaaaaaaaa             aa
1             aaahhhh             aa
2           abortions       abortion
3                abou            abo
4            abourted        aborted
5                 abs             ab
6                 ack           jack
7               acros         acrose
8             actualy         actual
9               adams           adam
10               adds            add
11            adduser          adder
12            adjusts         adjust
13             adoted        adopted
14         adreniline     adrenaline
15                ads             as
16             adults          adult
17                afe             fe
18            affairs         affair
19             affari       saffarid
20            affects         affect
21                afk             ak
22             agaibn          again
23               ages            age
24         aggravated      aggravate
25   agurlwithbigguns      lithobius
2